In [1]:
# coding:utf-8
import pandas as pd 
import pickle 
import pynlpir
import numpy as np
from collections import Counter
import jieba 
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.cluster import KMeans
import json

In [2]:
import xlrd
data = xlrd.open_workbook('data/知识点导出_2018-10-24.xlsx')  #读取数据
page = len(data.sheets())                             #获取sheet的数量
table = data.sheets()[0]                                #打开第一张表
nrows = table.nrows                                     #获取总行数
ncols = table.ncols                                       #获取总列数   
gold_data = []
knowledge = []
for i in range(1,nrows):
    know, ques = table.row_values(i)
    gold_data.append(ques)
    knowledge.append(know)

In [3]:
add_data = open('data/msgwyeth.txt', 'r').read().split('\n')
print knowledge[0]
len(add_data),len(knowledge)

你好


(501393, 1750)

In [4]:
def get_question(data, knowledge):
    questions = []
    knowledges = []
    for i in range(len(data)):
        ques = data[i].split('||')
        for que in ques:
            if que != '':
                questions.append(que)
                knowledges.append(knowledge[i])
    print questions[0]
    print len(questions)
    return questions, knowledges

In [5]:
len(gold_data), len(knowledge)

(1750, 1750)

In [6]:
gold_questions, knowledges = get_question(gold_data, knowledge)

亲在么
74936


In [7]:
len(knowledges)

74936

In [8]:
text = gold_questions#list(set(list(data['match'])))
len(text)
print len(text)
def jieba_tokenize(text):
    return jieba.lcut(text) 
 
tfidf_vectorizer = TfidfVectorizer(tokenizer=jieba_tokenize, \
lowercase=False)
'''
tokenizer: 指定分词函数
lowercase: 在分词之前将所有的文本转换成小写，因为涉及到中文文本处理，
所以最好是False
'''
text_list = text
 #需要进行聚类的文本集
tfidf_matrix = tfidf_vectorizer.fit_transform(text_list)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


74936


Loading model cost 0.411 seconds.
Prefix dict has been built succesfully.
/home/works/dl-tools/anaconda2/envs/tfenv.v1.2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [9]:
tfidf_matrix

<74936x16439 sparse matrix of type '<type 'numpy.float64'>'
	with 640871 stored elements in Compressed Sparse Row format>

In [10]:
'''
vector_arr = []
print len(sentences)
for i in range(len(sentences)):
    if i % 10000 == 0:print i
    vectors = []
    for j in range(len(sentences[i])):
        if sentences[i][j] in model:
            vectors.append(model[sentences[i][j]])
    final = []
    if len(vectors) == 0:
        final = [0.0] * 50
    else:
        for ii in range(len(vectors[0])):
            avr = 0
            for jj in range(len(vectors)):
                avr += vectors[jj][ii]
            avr = 1.0 * avr / len(vectors)
            final.append(avr)
    vector_arr.append(final)
X = np.array(vector_arr)
print X.shape
'''

'\nvector_arr = []\nprint len(sentences)\nfor i in range(len(sentences)):\n    if i % 10000 == 0:print i\n    vectors = []\n    for j in range(len(sentences[i])):\n        if sentences[i][j] in model:\n            vectors.append(model[sentences[i][j]])\n    final = []\n    if len(vectors) == 0:\n        final = [0.0] * 50\n    else:\n        for ii in range(len(vectors[0])):\n            avr = 0\n            for jj in range(len(vectors)):\n                avr += vectors[jj][ii]\n            avr = 1.0 * avr / len(vectors)\n            final.append(avr)\n    vector_arr.append(final)\nX = np.array(vector_arr)\nprint X.shape\n'

In [ ]:
import time
from sklearn.cluster import DBSCAN
start = time.time()
dbscan = DBSCAN(eps=.2) 
result = dbscan.fit_predict(tfidf_matrix) 
end = time.time()

In [ ]:
distance

In [13]:
result = clusterer.fit_predict(X)
print "Predicting result: ", result

Predicting result:  [-1 -1 -1 ... -1 -1 -1]


In [14]:
import pickle
pickle.dump(result, open('data/iter'+str(max_iter)+'.txt','w'))

NameError: name 'max_iter' is not defined

In [ ]:
result = pickle.load( open('data/iter'+str(300)+'.txt','r'))

In [22]:
new_real = []

In [23]:
df = pd.DataFrame({'request_word':text, 'predict':result, 'real': knowledges})
new_df = {}
new_df['class_num'] = []
new_df['request_words'] = []
c = 0
count = 0
new_texts = []
class_ = []
num_ = []
cat = []
for i in df['predict'].unique():
    d = list(df[df['predict'] == i]['request_word'])
    new_df['class_num'].append(c)
    new_df['request_words'].append('||'.join(d))
    new_texts.append('||'.join(d))
    num_.append(len(d))
    class_.append(d[0])
    cat.append(i)
    count += len(d)
    c += 1
dff = pd.DataFrame({'cat':cat, 'question':new_texts, 'class':class_, 'num':num_})
dff

,cat,class,num,question
0,-1,亲在么,69105,亲在么||亲在不？||亲在吗？|？？？？||亲在吗||亲，在？||亲，帮我看看一下||亲，在...
1,95,亲在吗？？,36,亲在吗？？||亲在吗？||亲，在吗？||亲，在吗?||亲，在吗||亲，在么||亲有人在吗|在...
2,400,亲好,137,亲好||节日快乐||咨询服务||早安||午安||午好||太謝谢||Thanks||8032...
3,12,亲，你好,11,亲，你好||亲|你好||亲|你好|亲|亲||你好北北老师||南南小姐姐你好||亲亲你好||您...
4,390,不好意思，昨晚太忙了,27,不好意思，昨晚太忙了||不好意思，我太忙了||晚安了，惠惠||晚安，不小点错了||没事了，晚...
5,370,亲爱的,20,亲爱的||清楚的||恩好的谢||上周五抽中的||歌诗达游轮的无限护照||适度水解蛋白的||真...
6,64,有人吗,23,有人吗||有人吗||机器人吗||没人吗||咨询电话免费吗||翻牌有礼要快递费吗||1%有小罐...
7,40,老师好,5,老师好||上午好||中午好||总有好多块||擦药不见好
8,55,你好[微笑],7,你好[微笑]||您好[愉快]||你好[愉快]||你好[玫瑰]||您好[微笑]||你好[握手]...
9,123,早[呲牙],638,早[呲牙]||亲[愉快][愉快][愉快]在不||早上好[愉快]||早上好[再见]||早上好[...


In [24]:
from collections import Counter
new_pred = []
#use = set()
map_ = {}
pred = list(dff['cat'])
for i in range(len(pred)):
    match = list(df[df['predict'] == pred[i]]['real']) #对每个聚类label找出对应问题真实值最多的作为他的label
    counter = Counter(match)
    all_pred = sorted(counter.items(), key=lambda item:item[1], reverse = True)
    #print all_pred  
    new_pred.append(all_pred[0][0])
    map_[pred[i]] = all_pred[0][0]

In [25]:
dff['new_pred'] = new_pred
dff

,cat,class,num,question,new_pred
0,-1,亲在么,69105,亲在么||亲在不？||亲在吗？|？？？？||亲在吗||亲，在？||亲，帮我看看一下||亲，在...,惠氏查真伪方法怎么验证1
1,95,亲在吗？？,36,亲在吗？？||亲在吗？||亲，在吗？||亲，在吗?||亲，在吗||亲，在么||亲有人在吗|在...,你好
2,400,亲好,137,亲好||节日快乐||咨询服务||早安||午安||午好||太謝谢||Thanks||8032...,删除
3,12,亲，你好,11,亲，你好||亲|你好||亲|你好|亲|亲||你好北北老师||南南小姐姐你好||亲亲你好||您...,你好
4,390,不好意思，昨晚太忙了,27,不好意思，昨晚太忙了||不好意思，我太忙了||晚安了，惠惠||晚安，不小点错了||没事了，晚...,点错了、发错了
5,370,亲爱的,20,亲爱的||清楚的||恩好的谢||上周五抽中的||歌诗达游轮的无限护照||适度水解蛋白的||真...,谢谢——1
6,64,有人吗,23,有人吗||有人吗||机器人吗||没人吗||咨询电话免费吗||翻牌有礼要快递费吗||1%有小罐...,注册版包装
7,40,老师好,5,老师好||上午好||中午好||总有好多块||擦药不见好,用药咨询
8,55,你好[微笑],7,你好[微笑]||您好[愉快]||你好[愉快]||你好[玫瑰]||您好[微笑]||你好[握手]...,你好
9,123,早[呲牙],638,早[呲牙]||亲[愉快][愉快][愉快]在不||早上好[愉快]||早上好[再见]||早上好[...,谢谢——1


In [26]:
pred = list(df['predict'])
new_pred = []
real = list(df['real'])
for i in range(len(pred)):
    new_pred.append(map_[pred[i]])

In [27]:
from sklearn import metrics
labels_true = real
labels_pred = new_pred
 
print (metrics.adjusted_rand_score(labels_true, labels_pred))
print (metrics.adjusted_mutual_info_score(labels_true, labels_pred) )

0.00029648547914451836
0.024994758607837238


In [ ]:
'''
print (metrics.fowlkes_mallows_score(labels_true, labels_pred))
print (metrics.homogeneity_score(labels_true, labels_pred))
print (metrics.completeness_score(labels_true, labels_pred))
print (metrics.v_measure_score(labels_true, labels_pred))
'''

In [ ]:
0.2546977431466231
0.4608132384545443